In [1]:
import os
import torch
from torch import nn, einsum
import numpy as np
import matplotlib.pyplot as plt
from einops import rearrange
import torch.nn.functional as F
import math
from fairscale.nn.checkpoint.checkpoint_activations import checkpoint_wrapper

In [2]:
%load_ext autoreload

%autoreload

from Models.WeatherGFT import GFT
from utils.dataloader import load_data
from utils.losses import weighted_rmse, weighted_mae, calculate_metrics

/home/fa.buzaev/.conda/envs/my_py_env1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## dataloader

In [4]:
data_root = '/home/fratnikov/weather_bench/'
prediction_horizone = 72

batch_size = 8

In [ ]:
import os
import shutil
from pathlib import Path

def copy_weather_data():
    # Исходная и целевая директории
    source_root = '/home/fratnikov/weather_bench/1.40625deg'
    target_root = '/home/fa.buzaev/data_to_debug'
    
    # Годы для копирования
    target_years = ['2015', '2018']
    
    # Создаем корневую целевую директорию
    Path(target_root).mkdir(parents=True, exist_ok=True)
    
    # Проходим по всем поддиректориям в исходной директории
    for variable_dir in os.listdir(source_root):
        source_dir = os.path.join(source_root, variable_dir)
        
        # Пропускаем, если это не директория
        if not os.path.isdir(source_dir):
            continue
            
        # Создаем соответствующую директорию в целевой папке
        target_dir = os.path.join(target_root, variable_dir)
        Path(target_dir).mkdir(parents=True, exist_ok=True)
        
        # Копируем файлы только нужных годов
        for file in os.listdir(source_dir):
            if any(f'_{year}_1.40625deg.nc' in file for year in target_years):
                source_file = os.path.join(source_dir, file)
                target_file = os.path.join(target_dir, file)
                print(source_file)
                print(target_file)
                # shutil.copy2(source_file, target_file)
                print(f'Скопирован файл: {file}')

if __name__ == '__main__':
    try:
        copy_weather_data()
        print('Копирование завершено успешно')
    except Exception as e:
        print(f'Произошла ошибка при копировании: {str(e)}')

In [ ]:
dataloader_train, dataloader_vali, dataloader_test, mean, std = load_data(batch_size=batch_size,
                                                                          val_batch_size=batch_size,
                                                                          data_root=data_root,
                                                                          num_workers=10,
                                                                          data_split='1_40625', # Разрешение и размерная сетка
                                                                          # data_split='5_625',
                                                                          data_name='mv_gft', # Название данных
                                                                          train_time=['2018', '2018'],
                                                                          # train_time=['2015', '2015'],
                                                                          # val_time=['2016', '2016'],
                                                                          # test_time=['2018', '2018'],
                                                                          val_time=None,
                                                                          test_time=None,
                                                                          idx_in=[0], # Размерность по T
                                                                          idx_out=[*range(1, prediction_horizone+1)],
                                                                          step=1,
                                                                          levels='all', 
                                                                          distributed=False, use_augment=False,
                                                                          use_prefetcher=False, drop_last=False)

In [9]:

test_iterator = iter(dataloader_train)

x_test, y_test = next(test_iterator)
x_test, y_test = x_test.to(device), y_test.to(device)

In [12]:
mean = torch.Tensor(mean).unsqueeze(0)  # [1, 1, 69, 1, 1]
std = torch.Tensor(std).unsqueeze(0)    # [1, 1, 69, 1, 1]

mean = mean.to(device)
std = std.to(device)

# Денормализация
x_test = x_test * std + mean
y_test = y_test * std + mean

Эта фигня нужна была только для того, чтобы взять std и mean, с которыми модель обучалась

In [ ]:
import json

with open('example_data/mean_std.json') as f:
    d = json.load(f)
    print(d)

In [17]:
for idx in range(x_test.shape[2]):
    x_test[:, :, idx] = (x_test[:, :, idx] - d['mean'][idx]) / d['std'][idx]

In [18]:
for idx in range(x_test.shape[2]):
    y_test[:, :, idx] = (y_test[:, :, idx] - d['mean'][idx]) / d['std'][idx]

In [20]:
model = GFT(hidden_dim=256,
            encoder_layers=[2, 2, 2],
            edcoder_heads=[3, 6, 6],
            encoder_scaling_factors=[0.5, 0.5, 1], # [128, 256] --> [64, 128] --> [32, 64] --> [32, 64], that is, patch size = 4 (128/32)
            encoder_dim_factors=[-1, 2, 2],

            body_layers=[4, 4, 4, 4, 4, 4], # A total of 4x6=24 HybridBlock, corresponding to 6 hours (24x15min) of time evolution
            body_heads=[8, 8, 8, 8, 8, 8],
            body_scaling_factors=[1, 1, 1, 1, 1, 1],
            body_dim_factors=[1, 1, 1, 1, 1, 1],

            decoder_layers=[2, 2, 2],
            decoder_heads=[6, 6, 3],
            decoder_scaling_factors=[1, 2, 1],
            decoder_dim_factors=[1, 0.5, 1],

            channels=69,
            head_dim=128,
            window_size=[4,8],
            relative_pos_embedding=False,
            out_kernel=[2,2],

            pde_block_depth=3, # 1 HybridBlock contains 3 PDE kernels, corresponding to 15 minutes (3x300s) of time evolution
            block_dt=300, # One PDE kernel corresponds to 300s of time evolution
            inverse_time=False).to(device)

In [ ]:
if os.path.exists('checkpoints/gft.ckpt'):
    ckpt = torch.load('checkpoints/gft.ckpt', map_location=device)
    model.load_state_dict(ckpt, strict=True)
    print('[complete loading model]')

In [ ]:
# Считаем количество параметров
total_params = sum(p.numel() for p in model.parameters())

print(f'Количество параметров в модели: {total_params}')

In [23]:
# Извлекаем веса всех слоев
weights = []
for param in model.parameters():
    if param.requires_grad:  # Это означает, что параметр обновляется во время обучения
        weights.append(param.data.cpu().numpy())  # Переводим в numpy для удобства

# Объединяем все веса в один массив для построения гистограммы
all_weights = np.concatenate([w.flatten() for w in weights])

In [ ]:
# Строим гистограмму
plt.figure(figsize=(8, 6))
plt.hist(all_weights, bins=1000, color='blue', alpha=0.7)
plt.title('Distribution of Weights in the Model')
plt.xlabel('Weight Value')
plt.ylabel('Frequency')
plt.grid(True)
plt.xlim(-0.2, 0.2)
plt.show()

In [ ]:
# outputs_full = torch.empty(x_test.shape, device=device)
outputs_full = np.empty([8, 72, 3, 69, 128, 256])
x_test_ = x_test[:, 0]
for j in range(72):
    with torch.no_grad():
        res = model(x_test_)
    outputs_full[:, j] = res.cpu().detach().numpy()
    x_test_ = res[:, 0]

In [ ]:
outputs_every_3 = np.empty([8, 72, 3, 69, 128, 256])  # размерность подгоните под задачу
x_test_ = x_test[:, 0]

for j in range(72):
    with torch.no_grad():
        
        res = model(x_test_)
        if j == 0:
            res_3 = res[:, 1]
            
        # Если j+1 кратно 3, обновляем x_test_ предсказанием с нулевого шага
        if (j + 1) % 3 == 0:
            x_test_ = res_3
            res_3 = res[:, 1]
        else:
            # Иначе продолжаем предсказывать на основе последнего результата
            x_test_ = res[:, 0]
        
    # Сохраняем предсказание для текущего шага
    outputs_every_3[:, j] = res.cpu().detach().numpy()

In [ ]:
outputs_every_6 = np.empty([8, 72, 3, 69, 128, 256])  # размерность подгоните под задачу
x_test_ = x_test[:, 0]

for j in range(72):
    with torch.no_grad():
        res = model(x_test_)
        
        if j == 0:
            res_3 = res[:, 1]
            res_6 = res[:, 2]  # сохраняем предсказание для шестого часа
    
    # Сохраняем предсказание для текущего шага
    outputs_every_6[:, j] = res.cpu().detach().numpy()
    
    # Обновляем x_test_ в зависимости от текущего часа
    if (j + 1) % 6 == 0:        # Каждые 6 часов
        x_test_ = res_6
        res_6 = res[:, 2]
    elif (j + 1) % 3 == 0:       # Каждые 3 часа (если не кратно 6)
        x_test_ = res_3
        res_3 = res[:, 1]
    else:                        # Остальные часы
        x_test_ = res[:, 0]

In [ ]:
outputs_every_6_2 = np.empty([8, 72, 3, 69, 128, 256], dtype=np.half)  # размерность подгоните под задачу
x_test_ = x_test[:, 0]

for j in range(72):
    with torch.no_grad():
        res = model(x_test_)
        
        if j == 0:
            res_6 = res[:, 2]  # сохраняем предсказание для шестого часа
    
    # Сохраняем предсказание для текущего шага
    outputs_every_6_2[:, j] = res.cpu().detach().numpy()
    
    # Обновляем x_test_ в зависимости от текущего часа
    if (j + 1) % 6 == 0:        # Каждые 6 часов
        x_test_ = res_6
        res_6 = res[:, 2]
    else:                        # Остальные часы
        x_test_ = res[:, 0]

In [29]:
def denorm(item, std, mean, idx=0):
    mean = mean[idx]
    std = std[idx]
    item_denorm = item * std + mean
    return item_denorm